**AI Summer Camp - Transfer Learning Projesi**

Bu projede “Cats and Dogs” veri setini kullanarak Transfer Learning ile derin öğrenme modeli eğiteceğiz. 

Veri Seti kaynağı olarak aşağıdaki kaynak kullanıldı.

https://www.microsoft.com/en-us/download/details.aspx?id=54765

Öncelikle veri setini indirip ekliyoruz.


In [8]:
!pip3 install gdown
!gdown --id "14ldbyr7HEmUcmRPDPm9Sh3j9qz2Cam5e" -O kagglecatsanddogs.zip
!unzip kagglecatsanddogs.zip -d ./kagglecatsanddogs/

Görüntülenen çıkış son 5000 satıra kısaltıldı.
  inflating: ./kagglecatsanddogs/PetImages/Dog/5500.jpg  
  inflating: ./kagglecatsanddogs/PetImages/Dog/5501.jpg  
  inflating: ./kagglecatsanddogs/PetImages/Dog/5502.jpg  
  inflating: ./kagglecatsanddogs/PetImages/Dog/5503.jpg  
  inflating: ./kagglecatsanddogs/PetImages/Dog/5504.jpg  
  inflating: ./kagglecatsanddogs/PetImages/Dog/5505.jpg  
  inflating: ./kagglecatsanddogs/PetImages/Dog/5506.jpg  
  inflating: ./kagglecatsanddogs/PetImages/Dog/5507.jpg  
  inflating: ./kagglecatsanddogs/PetImages/Dog/5508.jpg  
  inflating: ./kagglecatsanddogs/PetImages/Dog/5509.jpg  
  inflating: ./kagglecatsanddogs/PetImages/Dog/551.jpg  
  inflating: ./kagglecatsanddogs/PetImages/Dog/5510.jpg  
  inflating: ./kagglecatsanddogs/PetImages/Dog/5511.jpg  
  inflating: ./kagglecatsanddogs/PetImages/Dog/5512.jpg  
  inflating: ./kagglecatsanddogs/PetImages/Dog/5513.jpg  
  inflating: ./kagglecatsanddogs/PetImages/Dog/5514.jpg  
  inflating: ./kagglecatsa

Şimdi ise projede kullanılacak kütüphaneleri yüklüyoruz.


In [7]:
import os
import cv2
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
import tensorflow as tf
import pandas as pd
import numpy as np
import PIL
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import pathlib
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Dropout, Flatten,Activation, BatchNormalization,MaxPooling2D
from tensorflow.keras import datasets, layers, models
from os import path, listdir
from os.path import isdir
%matplotlib inline

Görüntülerin yeniden boyutlandırılma işlemini yapıyoruz. (resizing)
Bu işlemi yaparken hata veren görselleri ayrı listelere kaydediyoruz.

In [14]:
cats_names = os.listdir("kagglecatsanddogs/PetImages/Cat/")
dogs_names = os.listdir("kagglecatsanddogs/PetImages/Dog/")
sil_cats = []
sil_dogs = []


for f_name in cats_names:
  try:
    path = "kagglecatsanddogs/PetImages/Cat/" + f_name
    image = Image.open(path)
    new_image = image.resize((128, 128))
    new_path = "/content/new_resized/cats." + f_name
    new_image.save(new_path)
  except:
    print(f_name)
    sil_cats.append("/content/new_resized/cats." + f_name)
    continue

for f_name in dogs_names:
  try:
    path = "kagglecatsanddogs/PetImages/Dog/" + f_name
    image = Image.open(path)
    new_image = image.resize((128, 128))
    new_path = "/content/new_resized/dogs." + f_name
    new_image.save(new_path)
  except:
    print(f_name)
    sil_dogs.append("/content/new_resized/dogs." + f_name)
    continue

Görüntülenen çıkış son 5000 satıra kısaltıldı.
2670.jpg
4832.jpg
5761.jpg
4545.jpg
4915.jpg
1951.jpg
9497.jpg
9337.jpg
4889.jpg
3046.jpg
5164.jpg
626.jpg
6283.jpg
12389.jpg
3859.jpg
5318.jpg
1116.jpg
8994.jpg
6888.jpg
4817.jpg
9295.jpg
10162.jpg
2726.jpg
4822.jpg
3771.jpg
10902.jpg
1333.jpg
9407.jpg
352.jpg
1030.jpg
7117.jpg
416.jpg
3034.jpg
8673.jpg
6974.jpg
9507.jpg
2735.jpg
8595.jpg
8201.jpg
7433.jpg
6140.jpg
1972.jpg
9687.jpg
8413.jpg
3785.jpg
1598.jpg
813.jpg
1186.jpg
11383.jpg
5325.jpg
1614.jpg
8928.jpg
1779.jpg
10949.jpg
1631.jpg
349.jpg
7184.jpg
9259.jpg
2146.jpg
1574.jpg
412.jpg
7315.jpg
10836.jpg
6401.jpg
9757.jpg
2908.jpg
9854.jpg
9847.jpg
1506.jpg
12416.jpg
11831.jpg
9750.jpg
3715.jpg
7237.jpg
5574.jpg
8771.jpg
8532.jpg
17.jpg
551.jpg
11457.jpg
560.jpg
7732.jpg
11800.jpg
1272.jpg
6171.jpg
8384.jpg
3540.jpg
3190.jpg
10198.jpg
9451.jpg
3639.jpg
11099.jpg
11002.jpg
2477.jpg
7180.jpg
3378.jpg
12147.jpg
11756.jpg
3088.jpg
9944.jpg
10514.jpg
9914.jpg
203.jpg
4561.jpg
601.jpg
6518

Her bir görüntüyü etiketleyip listeye ekliyoruz.

In [ ]:
filenames = os.listdir("/content/new_resized/cats")

labels = []
input_path = []

for i in range(len(filenames)):
  input_path.append("/content/new_resized/cats" + filenames[i])
  file_name = filenames[i]
  label = file_name[0:4]

  if label == 'dogs':
    labels.append(0)
  else:
    labels.append(1)


Veri sayılarımızı kontrol ediyoruz.

In [ ]:
values, counts = np.unique(labels, return_counts=True)
print(values)
print(counts)

İsimleri ve etiketleri kullanarak dataframe oluşturuyoruz.

In [ ]:
df = pd.DataFrame()
df['images'] = input_path
df['label'] = labels
df = df.sample(frac=1).reset_index(drop=True)
df.head()

Görüntü klasörleri içinde hataya sebep olan ve ayrı listeye kaydedilen görüntülerin siliyoruz.


In [ ]:
df = df[df['images']!='/content/new_resized/dogs.Thumbs.db']
df = df[df['images']!='/content/new_resized/cats.Thumbs.db']

df.drop(df[df.images.isin(sil_cats)].index.tolist(), inplace=True)
df.drop(df[df.images.isin(sil_dogs)].index.tolist(), inplace=True)

Klasörlerdeki görüntü sayılarını görselleştiriyoruz.

In [ ]:
import seaborn as sns
sns.countplot(df['label'])

Oluşturduğumuz dataframe'deki etiket sütununu string data tipine dönüştürüyoruz.

In [ ]:
df['label'] = df['label'].astype('str')
df.head(10)

Data setini "train" ve "test" olarak ayırıyoruz.

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)

Normalizasyon işlemini gerçekleştiriyoruz.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
    rescale = 1./255,  # görsellerin normalize edilmesi
    rotation_range = 40, # overfittingin önlenmesi için görüntülerin büyütülmesi
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

val_generator = ImageDataGenerator(rescale = 1./255)

train_iterator = train_generator.flow_from_dataframe(
    train,x_col='images',
    y_col='label',
    target_size=(128,128),
    batch_size=512,
    class_mode='binary'
)

val_iterator = val_generator.flow_from_dataframe(
    test,x_col='images',
    y_col='label',
    target_size=(128,128),
    batch_size=512,
    class_mode='binary'
)

Modeli oluşturuyoruz.

In [ ]:
from keras import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense

model = Sequential([
            Conv2D(16, (3,3), activation='relu', input_shape=(128,128,3)),
            MaxPool2D((2,2)),
            Conv2D(32, (3,3), activation='relu'),
            MaxPool2D((2,2)),
            Conv2D(64, (3,3), activation='relu'),
            MaxPool2D((2,2)),
            Flatten(),
            Dense(512, activation='relu'),
            Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(train_iterator, epochs=10, validation_data=val_iterator)

Oluşturduğumuz modelin sonuçlarını görselleştiriyoruz.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))

plt.plot(epochs, acc, 'g', label='Training Accuracy')
plt.plot(epochs, val_acc, 'y', label='Validation Accuracy')
plt.title('Accuracy Graph')
plt.legend()
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(epochs, loss, 'g', label='Training Loss')
plt.plot(epochs, val_loss, 'y', label='Validation Loss')
plt.title('Loss Graph')
plt.legend()
plt.show()